첫 샘플의 베스트

"reindeer mistletoe elf gingerbread family advent scrooge chimney fireplace ornament"

# Genetic Algorithm

## Imports

In [144]:
import gc ## for memory
import os
import copy
from math import exp
from collections import Counter
from typing import List, Optional, Union

import numpy as np
import itertools
import pandas as pd
import transformers
import torch

from kaggle_evaluate import PerplexityCalculator

In [2]:
df_sample = pd.read_csv("sample_submission.csv")

In [3]:
df_sample.text.str.split().map(lambda x : len(set(x)))

0    10
1    20
2    20
3    29
4    50
5    89
Name: text, dtype: int64

## Setting

In [4]:
evaluatr = PerplexityCalculator("google/gemma-2-9b")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

## Algorithm

`-` Target

In [18]:
samples = df_sample.text[1:6].to_list()
samples

['advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and',
 'yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice',
 'yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice sing cheer and of the is eat visit relax unwrap',
 'hohoho candle poinsettia snowglobe peppermint eggnog fruitcake chocolate candy puzzle game doll toy workshop wonder believe dream hope peace joy merry season greeting card wrapping paper bow fireplace night cookie milk star wish wreath angel the to of and in that have it not with as you from we kaggle',
 'advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and yuletide decorations gifts cheer holiday carol magi nutcrac

`-` Code

In [137]:
## initiate
initial_times = 1000
genomes = ["" for _ in range(initial_times)]

for i in range(initial_times) :
    genomes[i] = np.random.permutation(samples[0].split())
    
genomes = np.array(genomes)

## selection
perplexities = np.array(evaluatr.get_perplexity([" ".join(genome) for genome in genomes], batch_size = 1024))
per_sum = sum(1/perplexities)
proba = 1/perplexities/per_sum

parents_indx = np.random.choice([i for i in range(initial_times)], p = proba, size = 10, replace = False)

## crossover


In [181]:
parents_proba = perplexities[parents_indx]
per_sum = sum(parents_proba)
proba = parents_proba/per_sum

cross_area = [[int(len(s.split())/4), int(len(s.split())/2), len(s.split())] for s in samples]

* 해당 영역에 공통으로 포함된 토큰을 배제 + 중복되는 토큰 처리
* 해당 영역의 반대쪽 부모에게서만 있는 토큰을 매핑하여 지정 + 중복되는 토큰 처리
* 공란에 나머지 토큰들을 순서대로 삽입

중복되는 토큰들은 뒤에 숫자를 매겨서(더미 토큰) 유니크하게 만든 다음에 따로 관리하는 게 좋지 않을까? -> 최적화 이슈는 있지만 딱히 상관없고, 구현이 쉬움

In [220]:
for t in set(samples[4].split()) :
    times = sum(np.array(samples[4].split()) == t)
    rep = 1
    
    if times > 1 :
        for i, k in enumerate(samples[4].split()) :          
            if k == t :
                tmp[i] = f"{t}{rep}"
                rep += 1
                
for i, t in enumerate(tmp) :
    try :
        int(t[-1])
        tmp[i] = t[:-1]
    except :
        pass

In [113]:
def crossover(parents_index : List[int], dupl = False) -> int :
    pair_parents = list(itertools.combinations(parents_indx, 2))
    
    for pair in pair_parents :
        parents = [genomes[pair[0]], genomes[pair[1]]]
        childs = []
        
        if dupl :
            for i in range(2) :
                for t in set(parents[i].split()) :
                    times = sum(np.array(parents[i].split()) == t)
                    rep = 1
                    
                    if times > 1 :
                        for j, k in enumerate(parents[i].split()) :
                            if k == t :
                                parents[i][j] = f"{t}{rep}"
                                rep += 1
        
        # PMX
        for i in range(2) :
            for j in range(100) :
                if np.random.random() < 0.2 :
                    width = np.random.randint(cross_area[0][0], cross_area[0][2])
                    
                else :
                    width = np.random.randint(cross_area[0][0], cross_area[0][1])
                    
                start = np.random.randint(0, cross_area[0][2]-width)
                child = [" " for k in range(cross_area[0][2])]
                
                parents
                
                child[start:start+width] = parents[i][start:start+width]
        
        if dupl :
            for i in range(2) :
                for j, t in enumerate(parents[i]) :
                    try :
                        int(t[-1])
                        parents[i][j] = t[:-1]
                    except :
                        pass
    
    return " ".join(child)

array(['mistletoe advent ornament family jump elf laugh drive walk give gingerbread bake the scrooge night chimney reindeer and fireplace sleep',
       'bake gingerbread mistletoe fireplace ornament reindeer family the and advent scrooge laugh sleep walk chimney drive night elf give jump',
       'give laugh elf advent walk reindeer the mistletoe sleep fireplace drive jump chimney bake night gingerbread ornament family and scrooge',
       'ornament give reindeer chimney sleep gingerbread jump and scrooge mistletoe advent fireplace family laugh walk bake elf the night drive',
       'sleep laugh drive chimney jump night advent family walk ornament give mistletoe the gingerbread scrooge reindeer fireplace elf and bake',
       'mistletoe laugh family drive bake sleep fireplace ornament gingerbread chimney advent elf and give reindeer scrooge walk the night jump',
       'mistletoe the laugh advent drive bake ornament family walk gingerbread sleep night reindeer fireplace scrooge jump g

## Submission

In [ ]:
df_submission = pd.DataFrame({"id":[0,1,2,3,4,5], "text":[" ".join(w) for w in best_word]})
df_submission.iloc[0, 1] = "reindeer mistletoe elf gingerbread family advent scrooge chimney fireplace ornament"
df_submission.to_csv("beam.csv")

os.system("kaggle competitions submit -c santa-2024 -f beam.csv -m .")